# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [7]:
# Load model

from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "openai-community/gpt2"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


# Set the attention mask
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Set the pad token ID to the EOS token ID
model.config.pad_token_id = tokenizer.eos_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Load rotten_tomatoes dataset
# See: https://huggingface.co/datasets/rotten_tomatoes

from datasets import load_dataset


train_dataset = load_dataset("rotten_tomatoes", split='train[:500]')
test_dataset = load_dataset("rotten_tomatoes", split='test[:100]')

# View the dataset characteristics
print(train_dataset)
print(test_dataset)

# Tokenize the dataset
train_dataset_tokenized = train_dataset.map(
    lambda x: tokenizer(x["text"], truncation=True), batched=True
)

test_dataset_tokenized = test_dataset.map(
    lambda x: tokenizer(x["text"], truncation=True), batched=True
)

print(train_dataset_tokenized[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})
Dataset({
    features: ['text', 'label'],
    num_rows: 100
})


Map: 100%|██████████| 100/100 [00:00<00:00, 4763.66 examples/s]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1, 'input_ids': [1169, 3881, 318, 23985, 284, 307, 262, 2310, 301, 4289, 338, 649, 366, 369, 272, 366, 290, 326, 339, 338, 1016, 284, 787, 257, 22870, 772, 3744, 621, 610, 77, 727, 5513, 5767, 89, 44028, 837, 474, 11025, 12, 565, 3885, 5719, 1801, 1326, 393, 2876, 574, 384, 13528, 764], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Show dataset example entries

In [13]:
for i in range(10):
    print(test_dataset[i])

{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .', 'label': 1}
{'text': 'consistently clever and suspenseful .', 'label': 1}
{'text': 'it\'s like a " big chill " reunion of the baader-meinhof gang , only these guys are more harmless pranksters than political activists .', 'label': 1}
{'text': 'the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .', 'label': 1}
{'text': 'red dragon " never cuts corners .', 'label': 1}
{'text': 'fresnadillo has something serious to say about the ways in which extravagant chance can distort our perspective and throw us off the path of good sense .', 'label': 1}
{'text': 'throws in enough clever and unexpected twists to make the formula feel fresh .', 'label': 1}
{'text': 'weighty and ponderous but every bit as filling as the treat of the title .', 'label': 1}
{'text': "a real audience

Validate pretrained model on the Rotten Tomatoes dataset

In [11]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
    ),
    eval_dataset=test_dataset_tokenized,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# evaluate the foundational model
evaluation_results = trainer.evaluate()
print(evaluation_results)

100%|██████████| 13/13 [00:04<00:00,  2.66it/s]

{'eval_loss': 1.1482954025268555, 'eval_accuracy': 0.0, 'eval_runtime': 5.2249, 'eval_samples_per_second': 19.139, 'eval_steps_per_second': 2.488}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.